In [1]:
#Pre processing of the data
#Loading the dataset
import pandas as pd
df=pd.read_csv("D:/UOFG/SEM2/IV/Coursework/SystemB/imdb1000moviesinjune2022.csv")
df['Year'] = df['Year'].str.replace('-', '')
df['Year'] = df['Year'].str.replace('(', '')
df['Year'] = df['Year'].str.replace(')', '')
df['Year'] = df['Year'].str.replace('I', '')
df['GROSS'] = df['GROSS'].str.replace('M', '')
df['GROSS'] = df['GROSS'].str.replace('$', '')
df.dropna(inplace=True)
df['genre_new'] = df['genre'].str.split(',').str[0]
df['Year'] = df['Year'].astype(int)
df['GROSS'] = (df['GROSS'].astype(float)
                             .round(0)
                             .astype(int))
def categorize_year(Year):
    if Year >= 1900:
        decade_start = (Year // 10) * 10
        decade_end = decade_start + 9
        return f"{decade_start}-{decade_end}"
    else:
        return "Before 1900"

df['Decade'] = df['Year'].apply(categorize_year)
df[['Year', 'Decade']].head()

df.to_csv('D:/UOFG/SEM2/IV/Coursework/SystemB/modified_data.csv', index=False)

C:\Users\abhij\AppData\Local\Temp\ipykernel_18032\2393940737.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace('(', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_18032\2393940737.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year'] = df['Year'].str.replace(')', '')
C:\Users\abhij\AppData\Local\Temp\ipykernel_18032\2393940737.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['GROSS'] = df['GROSS'].str.replace('$', '')


In [2]:
#System B


import altair as alt
import pandas as pd

# Load the dataset
df = pd.read_csv('D:/UOFG/SEM2/IV/Coursework/SystemB/modified_Data.csv')

# Define a selection for brushing and linking
brush = alt.selection(type='interval', encodings=['x'])

# Define the color scheme
color_scheme = alt.Scale(scheme='tableau20')

# Bar chart of average gross earnings by genre with interactive brushing
bar_chart = alt.Chart(df).mark_bar().encode(
    alt.X('genre_new:N', title='Genre'),
    alt.Y('average(GROSS):Q', title='Average Gross Earnings(in million)'),
    color=alt.condition(brush, alt.Color('genre_new:N', scale=color_scheme), alt.value('lightgray')),
    tooltip=['genre_new', 'average(GROSS)']
).properties(
    title='Average Gross Earnings by Genre',
    width=300,
    height=300
).add_selection(
    brush
)

# Heat map of average metascore by genre over time with brushing and linking
heat_map = alt.Chart(df).mark_rect().encode(
    alt.X('Decade:O', title='Year'),
    alt.Y('genre_new:N', title='Genre'),
    color=alt.condition(brush, 'average(metascore):Q', alt.value('lightgray')),  # Removed custom scale
    tooltip=['Decade', 'genre_new', alt.Tooltip('average(metascore)', title='Average Metascore')]
).properties(
    title='Average Metascore by Genre Over Time',
    width=300,
    height=300
).add_selection(
    brush
)

stacked_bar_chart = alt.Chart(df).mark_bar().encode(
    alt.Y('certificate:N', title='Certificate'),
    alt.X('sum(GROSS):Q', title='Gross Earnings(in million)'),
    color=alt.Color('genre_new:N', scale=color_scheme),
    tooltip=['certificate', 'genre_new', alt.Tooltip('sum(GROSS)', title='Gross Earnings(in million)')]
).properties(
    title='Gross Earnings by Movie Certificate',
    width=300,
    height=300
).add_selection(
    brush
)


text_overlay = alt.Chart(df).mark_text(align='left', baseline='middle', dx=3).encode(
    alt.Y('certificate:N'),
    alt.X('sum(GROSS):Q', stack='zero'),
    text=alt.Text('sum(GROSS):Q'),
    color=alt.value('black')  
).transform_filter(
    brush
)

final_stacked_bar_chart = (stacked_bar_chart + text_overlay).resolve_scale(color='independent')
final_chart = alt.hconcat(bar_chart, heat_map, final_stacked_bar_chart).resolve_scale(y='independent')
final_chart.save('D:/UOFG/SEM2/IV/Coursework/SystemB/sysB.html')


D:\Anaconda\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
D:\Anaconda\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
